In [1]:
import pandas as pd
import pickle
from functools import reduce

import fpl_methods as methods

with open ('season_data.pickle','rb') as file:
    data = pickle.load(file)


current_gw = max(data["managers"][0].picks)

managers = data["managers"]
players = data["players"]
transfers = data["transfers"]
team = data["teams"]

findings = {}

### GW Standings & Positional Standings 

In [2]:
ranking_targets = ["total", "gk", "def", "mid", "fwd", "bench", "missed", "points_against", "points_difference"]
for target in ranking_targets:
    findings[target + " gw"] = methods.get_ranking_gw(managers,"points",target)
    
    findings[target] = methods.get_ranking(managers,"points",target)

In [3]:
standing_fields = ["h2h_points","wins","draws","losses"]
for field in standing_fields:
    findings[field] = methods.get_ranking(managers,field)

In [4]:
findings["total"]

,manager_short_name,manager,points
0,YE,Youssef E,1347
1,SB,Sandipan B,1318
2,YA,Youssef A,1290
3,SL,Sami L,1241
4,pa,peter a,1205
5,ST,Shrey T,1182
6,YA1,Yahya A,1179
7,ES,Ege S,1143
8,SS,Santi S,1141
9,WN,Will N,1116


### General Standings Table

In [5]:
standings_df = pd.DataFrame(columns=["manager_short","manager","h2h_pts","wins","draws","losses","pts"])

In [6]:
points = findings["h2h_points"]
wins = findings["wins"]
draws = findings["draws"]
losses = findings["losses"]

data_frames = [points, wins, draws, losses]
standings_table = reduce(lambda left, right: pd.merge(left, right, on=['manager_short_name', 'manager']), data_frames)

In [7]:
standings_table

,manager_short_name,manager,h2h_points,wins,draws,losses
0,YE,Youssef E,73,24,1,8
1,YA,Youssef A,65,21,2,10
2,pa,peter a,63,21,0,12
3,SB,Sandipan B,60,20,0,13
4,WN,Will N,48,16,0,17
5,ES,Ege S,48,16,0,17
6,SL,Sami L,46,15,1,17
7,ST,Shrey T,46,15,1,17
8,SS,Santi S,45,15,0,18
9,AB,Anitej B,45,15,0,18


### H2H

In [8]:
data["managers"][0].fixtures.items()

dict_items([(1, {'result': 'L', 'points': 34, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA0474340>, 'opponent_points': 40}), (2, {'result': 'L', 'points': 20, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA0474580>, 'opponent_points': 56}), (3, {'result': 'L', 'points': 20, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA04743A0>, 'opponent_points': 45}), (4, {'result': 'W', 'points': 43, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA0474550>, 'opponent_points': 20}), (5, {'result': 'L', 'points': 24, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA0474400>, 'opponent_points': 47}), (6, {'result': 'L', 'points': 29, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA04744F0>, 'opponent_points': 36}), (7, {'result': 'W', 'points': 31, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA0474460>, 'opponent_points': 28}), (8, {'result': 'W', 'points': 33, 'opponent': <fpl_classes.FPLmanager object at 0x0000019BA0474490>, 'oppo

In [9]:
h2h_dfs = {}

for manager in managers:
    h2h_fix = pd.DataFrame()
    for gw, match in manager.fixtures.items():
        match["manager"] = manager.short_name
        row = pd.DataFrame(match,index=[len(h2h_fix)+1])
        row["opponent"] = match["opponent"].short_name
        if gw == 1:
            h2h_fix = row
        else:
            h2h_fix = pd.concat([h2h_fix,row])
        
    h2h_fix["point_diff"] = h2h_fix["points"] - h2h_fix["opponent_points"]
    h2h_dfs[manager.short_name] = h2h_fix

In [10]:
h2h_dfs["SL"].head()

,result,points,opponent,opponent_points,manager,point_diff
1,L,37,SB,47,SL,-10
2,W,56,AA,20,SL,36
3,W,30,RK,16,SL,14
4,L,27,WN,33,SL,-6
5,D,36,ST,36,SL,0


##### Average Win, Best Win, Closest Win

In [11]:
df = h2h_dfs["SS"]
win_avg_pts = df.loc[df["result"]=="W"]["points"].mean()
win_avg_opp_pts = df.loc[df["result"]=="W"]["opponent_points"].mean()
print("In the average win, you had", round(win_avg_pts,2), "and they had", round(win_avg_opp_pts,2))

In the average win, you had 40.56 and they had 29.5


In [12]:
df.loc[df["point_diff"]==max(df["point_diff"])]

,result,points,opponent,opponent_points,manager,point_diff
29,W,60,KS,26,SS,34


In [13]:
wins = df.loc[df["result"]=="W"]
wins.loc[wins["point_diff"]==min(wins["point_diff"])]

,result,points,opponent,opponent_points,manager,point_diff
23,W,29,AA,27,SS,2


##### Average loss, Best loss, Closest loss

In [14]:
loss_avg_pts = df.loc[df["result"]=="L"]["points"].mean()
loss_avg_opp_pts = df.loc[df["result"]=="L"]["opponent_points"].mean()
print("In the average loss, you had", round(loss_avg_pts,2), "and they had", round(loss_avg_opp_pts,2))

In the average loss, you had 30.89 and they had 47.0


In [15]:
df.loc[df["point_diff"]==min(df["point_diff"])]

,result,points,opponent,opponent_points,manager,point_diff
5,L,19,SB,55,SS,-36


In [16]:
losses = df.loc[df["result"]=="L"]
losses.loc[losses["point_diff"]==max(losses["point_diff"])]

,result,points,opponent,opponent_points,manager,point_diff
16,L,26,KS,29,SS,-3
21,L,31,WN,34,SS,-3


In [17]:
for man in h2h_dfs.keys():
    
    print(man, "min:", min(h2h_dfs[man]["point_diff"]), "max:", max(h2h_dfs[man]["point_diff"]))

AA min: -36 max: 34
SL min: -35 max: 41
SB min: -23 max: 38
WN min: -31 max: 22
RK min: -37 max: 28
SS min: -36 max: 34
ST min: -22 max: 24
AB min: -30 max: 18
YE min: -41 max: 37
pa min: -31 max: 27
YA min: -20 max: 34
ES min: -38 max: 28
YA1 min: -19 max: 35
KS min: -34 max: 23


In [18]:
h2h_fix.groupby(["opponent","result"]).size().sort_values(ascending=False)

opponent  result
AB        L         3
ST        L         3
SS        W         2
pa        L         2
YE        W         2
RK        L         2
SB        W         2
SL        L         2
YA        D         2
WN        L         2
          W         1
YE        L         1
YA1       W         1
          L         1
          D         1
YA        L         1
AA        D         1
          L         1
SS        L         1
SB        L         1
RK        W         1
ES        W         1
          L         1
          D         1
AA        W         1
pa        W         1
dtype: int64

### Stats

In [19]:
targets = ["goals_scored","assists","clean_sheets","goals_conceded","own_goals","penalties_saved","penalties_missed",
          "yellow_cards","red_cards","saves","bonus","dreamteam"]
picks = managers[3].picks

for target in targets:
    findings[target] = methods.get_stat_ranking(managers, targets[0])
    

In [20]:
list(findings.keys())

['total gw',
 'total',
 'gk gw',
 'gk',
 'def gw',
 'def',
 'mid gw',
 'mid',
 'fwd gw',
 'fwd',
 'bench gw',
 'bench',
 'missed gw',
 'missed',
 'points_against gw',
 'points_against',
 'points_difference gw',
 'points_difference',
 'h2h_points',
 'wins',
 'draws',
 'losses',
 'goals_scored',
 'assists',
 'clean_sheets',
 'goals_conceded',
 'own_goals',
 'penalties_saved',
 'penalties_missed',
 'yellow_cards',
 'red_cards',
 'saves',
 'bonus',
 'dreamteam']

### Transfers

##### Most Transfers

In [21]:
df = pd.DataFrame(columns=["manager_short_name","manager","num_transfers"])
for man in managers:
    transfer_count = 0
    for transfer in man.transfers:
        if transfer.result == "a":
            transfer_count+=1
        
    df.loc[df.shape[0]] = {"manager_short_name":man.short_name,"manager":man.name,"num_transfers": transfer_count}

df.sort_values("num_transfers", ascending=False).reset_index(drop=True)

,manager_short_name,manager,num_transfers
0,AA,Ali,70
1,AB,Anitej,60
2,SB,Sandipan,57
3,SS,Santi,43
4,ES,Ege,37
5,SL,Sami,36
6,KS,Kian,29
7,YE,Youssef,19
8,ST,Shrey,16
9,WN,Will,14


##### Most Transferred In

In [22]:
df = pd.DataFrame(columns=["manager_short_name","manager","player","num_transfers"])
for man in managers:
    for tran in man.transfers:
        if tran.result == "a":
            if tran.player_in.name in df["player"].values:
                df.loc[df["player"] == tran.player_in.name,"num_transfers"] += 1
            else:
                df.loc[df.shape[0]] = {"manager_short_name":man.short_name,"manager":man.name,"player":tran.player_in.name, "num_transfers": 1}

df = df.loc[df["num_transfers"]>1].sort_values("num_transfers",ascending=False).reset_index(drop=True)
df[:20]

,manager_short_name,manager,player,num_transfers
0,SL,Sami,Tavernier,5
1,AA,Ali,Neto,4
2,AA,Ali,Lamptey,4
3,AA,Ali,Bradley,4
4,SL,Sami,Gusto,4
5,AA,Ali,Livramento,4
6,SB,Sandipan,Tarkowski,4
7,AA,Ali,Trippier,4
8,AA,Ali,McNeil,4
9,SB,Sandipan,Beto,4


In [23]:
managers[0].transfers[0].player_in.name

'Mazraoui'

##### Best & Worst Transfers

In [24]:
managers[0].transfers[0].gameweek
player_name = managers[0].transfers[0].player_in.name

In [25]:
player_name

'Mazraoui'

In [ ]:
man.picks

{1: {'pos 1': <fpl_classes.FPLplayer at 0x1f154c2c130>,
  'pos 2': <fpl_classes.FPLplayer at 0x1f1534bfe50>,
  'pos 3': <fpl_classes.FPLplayer at 0x1f15337b670>,
  'pos 4': <fpl_classes.FPLplayer at 0x1f154c2e320>,
  'pos 5': <fpl_classes.FPLplayer at 0x1f154c2de70>,
  'pos 6': <fpl_classes.FPLplayer at 0x1f154c2d630>,
  'pos 7': <fpl_classes.FPLplayer at 0x1f1534bfb50>,
  'pos 8': <fpl_classes.FPLplayer at 0x1f154c2cfd0>,
  'pos 9': <fpl_classes.FPLplayer at 0x1f154c2e1d0>,
  'pos 10': <fpl_classes.FPLplayer at 0x1f15337b8e0>,
  'pos 11': <fpl_classes.FPLplayer at 0x1f154c2e230>,
  'pos 12': <fpl_classes.FPLplayer at 0x1f1534bfdc0>,
  'pos 13': <fpl_classes.FPLplayer at 0x1f154c2d240>,
  'pos 14': <fpl_classes.FPLplayer at 0x1f154c2d540>,
  'pos 15': <fpl_classes.FPLplayer at 0x1f1534bc790>,
  'formation': [3, 5, 1]},
 2: {'pos 1': <fpl_classes.FPLplayer at 0x1f154c2c130>,
  'pos 2': <fpl_classes.FPLplayer at 0x1f15337b670>,
  'pos 3': <fpl_classes.FPLplayer at 0x1f1534bfe50>,
  'pos 

In [ ]:
df = pd.DataFrame(columns=["manager_short_name","manager","gameweek","player_in","player_out","gws","in_pts","out_pts","pts_gain"])
current_gw = 32
for man in managers:                                            # for every manager
    for tran in man.transfers:                                  # for every one of their transfers
        in_pts = 0
        out_pts = 0
        gws = 0

        if tran.result == "a":                                  # for every accepted transfer
            tran_gw = tran.gameweek
            player_in = tran.player_in
            player_out = tran.player_out

            for gw in range(tran_gw,min(tran_gw+3,current_gw)): # for the next three gws after the transfer
                position = 0
                for pos, player in man.picks[gw].items():       # check all the players in the manager's 11
                    position += 1
                    if position <= 15:
                        if player == player_in:
                            gws += 1
                            in_pts += player.points[gw]
                            out_pts += player_out.points[gw]
            row = {"manager_short_name":man.short_name,"manager":man.name, "gameweek":tran_gw,"player_in":player_in.name, "player_out":player_out.name,"gws":gws,"in_pts":in_pts,"out_pts":out_pts,"pts_gain":in_pts-out_pts}
            df.loc[df.shape[0]] = row

df.sort_values("pts_gain")

,manager_short_name,manager,gameweek,player_in,player_out,gws,in_pts,out_pts,pts_gain
235,ST,Shrey,14,Werner,J.Murphy,3,3,22,-19
314,YE,Youssef,6,Tosin,Aït-Nouri,3,0,19,-19
328,pa,peter,4,Kerkez,De Ligt,3,4,22,-18
170,WN,Will,10,Beto,Armstrong,3,3,16,-13
416,KS,Kian,29,Todibo,Gusto,3,2,14,-12
...,...,...,...,...,...,...,...,...,...
114,SB,Sandipan,5,McNeil,Odobert,3,19,0,19
215,SS,Santi,24,Marmoush,Ings,3,21,1,20
323,YE,Youssef,16,Huijsen,White,3,24,0,24
27,AA,Ali,11,Amad,Carvalho,3,25,1,24
